# Prueba de Hipótesis Geoespaciales
## MIT LIFT Lab - UNSAM - UBA

### Hipótesis a probar:
1. **H1**: Crimen alto + Sin crédito → Menor expectativa de crecimiento
2. **H2**: Crimen bajo + Con crédito → Mayor inversión tecnológica

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, mannwhitneyu
import json

# Cargar datos
df = pd.read_csv('datos_comercios.csv')
print(f"Total de comercios: {len(df)}")
print(f"\nColumnas relevantes disponibles:")
print(df[['afect_crimen', 'afect_credito', 'quiere_crezca', 'tecnologia']].head())

## 1. Preparación de datos

In [ ]:
# Categorizar percepción de crimen
def categorizar_crimen(valor):
    if pd.isna(valor):
        return None
    if valor in ['Mucho']:
        return 'Alto'
    elif valor in ['Nada', 'Poco']:
        return 'Bajo'
    else:
        return 'Medio'

df['crimen_cat'] = df['afect_crimen'].apply(categorizar_crimen)

# Detectar acceso a crédito (cualquier fuente > 0)
credito_cols = ['credits_bancos', 'credits_proveedor', 'credits_familia', 'credits_gobierno', 'credits_privado']
df['tiene_credito'] = (df[credito_cols].sum(axis=1) > 0).astype(int)

# Categorizar tecnología
def categorizar_tech(valor):
    if pd.isna(valor):
        return None
    if 'Alto' in str(valor) or 'mayoría' in str(valor).lower():
        return 'Alto'
    elif 'Básico' in str(valor) or 'al menos una' in str(valor).lower():
        return 'Básico'
    else:
        return 'Moderado'

df['tech_cat'] = df['tecnologia'].apply(categorizar_tech)

print("\n=== DISTRIBUCIÓN DE VARIABLES ===")
print(f"\nCrimen:")
print(df['crimen_cat'].value_counts())
print(f"\nAcceso a crédito:")
print(df['tiene_credito'].value_counts())
print(f"\nIntención de crecer:")
print(df['quiere_crezca'].value_counts())
print(f"\nNivel tecnológico:")
print(df['tech_cat'].value_counts())

## 2. Hipótesis 1: Crimen alto + Sin crédito → Menor expectativa de crecimiento

In [ ]:
# Crear grupos
df_h1 = df[df['crimen_cat'].isin(['Alto', 'Bajo']) & df['quiere_crezca'].notna()].copy()

# Grupo 1: Crimen alto + Sin crédito
grupo1 = df_h1[(df_h1['crimen_cat'] == 'Alto') & (df_h1['tiene_credito'] == 0)]
# Grupo 2: Otros (para comparación)
grupo2 = df_h1[~((df_h1['crimen_cat'] == 'Alto') & (df_h1['tiene_credito'] == 0))]

print("\n=== HIPÓTESIS 1: Crimen alto + Sin crédito ===")
print(f"\nGrupo adverso (Crimen alto + Sin crédito): {len(grupo1)} comercios")
print(f"Intención de crecer: {grupo1['quiere_crezca'].mean():.1%}")
print(f"\nGrupo comparación: {len(grupo2)} comercios")
print(f"Intención de crecer: {grupo2['quiere_crezca'].mean():.1%}")

# Prueba estadística (Chi-cuadrado)
tabla_contingencia = pd.crosstab(df_h1['quiere_crezca'], 
                                  (df_h1['crimen_cat'] == 'Alto') & (df_h1['tiene_credito'] == 0))
chi2, p_valor, dof, expected = chi2_contingency(tabla_contingencia)

print(f"\n--- Prueba Chi-cuadrado ---")
print(f"Chi-cuadrado: {chi2:.4f}")
print(f"P-valor: {p_valor:.4f}")
print(f"Significancia: {'SÍ (p < 0.05)' if p_valor < 0.05 else 'NO (p >= 0.05)'}")

diferencia = grupo2['quiere_crezca'].mean() - grupo1['quiere_crezca'].mean()
print(f"\nDiferencia: {diferencia:.1%} {'menor' if diferencia > 0 else 'mayor'} intención de crecer en grupo adverso")

h1_resultado = {
    'grupo_adverso': {
        'n': int(len(grupo1)),
        'crecimiento_pct': float(grupo1['quiere_crezca'].mean() * 100)
    },
    'grupo_comparacion': {
        'n': int(len(grupo2)),
        'crecimiento_pct': float(grupo2['quiere_crezca'].mean() * 100)
    },
    'estadistica': {
        'chi2': float(chi2),
        'p_valor': float(p_valor),
        'significativo': bool(p_valor < 0.05)
    },
    'conclusion': 'CONFIRMADA' if (p_valor < 0.05 and diferencia > 0) else ('NO RECHAZADA' if p_valor >= 0.05 else 'RECHAZADA')
}

## 3. Hipótesis 2: Crimen bajo + Con crédito → Mayor inversión tecnológica

In [ ]:
# Crear grupos
df_h2 = df[df['crimen_cat'].isin(['Alto', 'Bajo']) & df['tech_cat'].notna()].copy()

# Asignar valores numéricos a tecnología
tech_map = {'Básico': 1, 'Moderado': 2, 'Alto': 3}
df_h2['tech_num'] = df_h2['tech_cat'].map(tech_map)

# Grupo 1: Crimen bajo + Con crédito
grupo1_h2 = df_h2[(df_h2['crimen_cat'] == 'Bajo') & (df_h2['tiene_credito'] == 1)]
# Grupo 2: Otros
grupo2_h2 = df_h2[~((df_h2['crimen_cat'] == 'Bajo') & (df_h2['tiene_credito'] == 1))]

print("\n=== HIPÓTESIS 2: Crimen bajo + Con crédito ===")
print(f"\nGrupo favorable (Crimen bajo + Con crédito): {len(grupo1_h2)} comercios")
print(f"Nivel tecnológico promedio: {grupo1_h2['tech_num'].mean():.2f}")
print(f"Distribución: {grupo1_h2['tech_cat'].value_counts().to_dict()}")
print(f"\nGrupo comparación: {len(grupo2_h2)} comercios")
print(f"Nivel tecnológico promedio: {grupo2_h2['tech_num'].mean():.2f}")
print(f"Distribución: {grupo2_h2['tech_cat'].value_counts().to_dict()}")

# Prueba Mann-Whitney U (no paramétrica)
statistic, p_valor_h2 = mannwhitneyu(grupo1_h2['tech_num'], grupo2_h2['tech_num'], alternative='greater')

print(f"\n--- Prueba Mann-Whitney U ---")
print(f"Estadístico U: {statistic:.4f}")
print(f"P-valor: {p_valor_h2:.4f}")
print(f"Significancia: {'SÍ (p < 0.05)' if p_valor_h2 < 0.05 else 'NO (p >= 0.05)'}")

diferencia_h2 = grupo1_h2['tech_num'].mean() - grupo2_h2['tech_num'].mean()
print(f"\nDiferencia: {diferencia_h2:.2f} puntos {'mayor' if diferencia_h2 > 0 else 'menor'} nivel tecnológico en grupo favorable")

# Calcular % con tecnología "Alta"
pct_alto_g1 = (grupo1_h2['tech_cat'] == 'Alto').mean() * 100
pct_alto_g2 = (grupo2_h2['tech_cat'] == 'Alto').mean() * 100

h2_resultado = {
    'grupo_favorable': {
        'n': int(len(grupo1_h2)),
        'tech_promedio': float(grupo1_h2['tech_num'].mean()),
        'tech_alta_pct': float(pct_alto_g1)
    },
    'grupo_comparacion': {
        'n': int(len(grupo2_h2)),
        'tech_promedio': float(grupo2_h2['tech_num'].mean()),
        'tech_alta_pct': float(pct_alto_g2)
    },
    'estadistica': {
        'mann_whitney_u': float(statistic),
        'p_valor': float(p_valor_h2),
        'significativo': bool(p_valor_h2 < 0.05)
    },
    'conclusion': 'CONFIRMADA' if (p_valor_h2 < 0.05 and diferencia_h2 > 0) else ('NO RECHAZADA' if p_valor_h2 >= 0.05 else 'RECHAZADA')
}

## 4. Conclusiones y Exportación

In [ ]:
resultados = {
    'hipotesis_1': h1_resultado,
    'hipotesis_2': h2_resultado,
    'resumen': {
        'h1_confirmada': h1_resultado['conclusion'] == 'CONFIRMADA',
        'h2_confirmada': h2_resultado['conclusion'] == 'CONFIRMADA'
    }
}

# Guardar resultados
with open('resultados_hipotesis.json', 'w', encoding='utf-8') as f:
    json.dump(resultados, f, indent=2, ensure_ascii=False)

print("\n" + "="*60)
print("CONCLUSIONES FINALES")
print("="*60)
print(f"\n✓ Hipótesis 1: {h1_resultado['conclusion']}")
print(f"  Comercios con crimen alto + sin crédito muestran {grupo1['quiere_crezca'].mean():.1%} intención de crecer")
print(f"  vs {grupo2['quiere_crezca'].mean():.1%} en otros comercios (diferencia: {abs(diferencia):.1%})")
print(f"\n✓ Hipótesis 2: {h2_resultado['conclusion']}")
print(f"  Comercios con crimen bajo + con crédito: {pct_alto_g1:.1f}% tiene tecnología alta")
print(f"  vs {pct_alto_g2:.1f}% en otros comercios (diferencia: {abs(pct_alto_g1 - pct_alto_g2):.1f}pp)")
print("\n" + "="*60)
print("Resultados exportados a: resultados_hipotesis.json")
print("="*60)